## **TITLE** --
 **Toxic Comment Classification using Machine Learning and NLP**

## **AIM**--
`The aim of this project is to build a machine learning model that can automatically classify textual comments as toxic or non-toxic using Natural Language Processing techniques`

## **PROBLEM STATEMENT**-



> Manual identification of toxic comments on online platforms is inefficient. This project aims to automate the detection of toxic comments using machine learning.



### OBJECTIVES--


* To preprocess textual data
* To extract meaningful features using TF-IDF
* To train a machine learning classifier
* To evaluate the model performance






## **TOOLS & TECHNOLOGIES**--

1. Python
2. Google Colab
3. Pandas, NumPy
4. NLTK
5. Scikit-learn


## **DATASET DESCRIPTION--**



> The dataset used in this project is the Jigsaw Toxic Comment Classification Dataset, which contains labeled Wikipedia comments.



**DATASET DOWNLOAD**

> The dataset is downloaded using KaggleHub directly into the Google Colab environment.




In [1]:
import kagglehub

path = kagglehub.dataset_download(
    "julian3833/jigsaw-toxic-comment-classification-challenge"
)


100%|██████████| 53.4M/53.4M [00:01<00:00, 29.5MB/s]

Extracting files...


In [2]:
# IMPORTING LIBRARIES--
import pandas as pd
import numpy as np
import re
import nltk


**The downloaded dataset is loaded into a pandas DataFrame for further processing.**

In [3]:
import os
train_path = os.path.join(path, "train.csv")
df = pd.read_csv(train_path)
df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


> **DATA CLEANING**


In [4]:
df = df[['comment_text', 'toxic']]
df.isnull().sum()


,0
comment_text,0
toxic,0


### TEXT PREPROCESSING--

**Text preprocessing is performed to remove noise and improve model performance**.

In [5]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return " ".join(text)

df['clean_comment'] = df['comment_text'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### FEATURES SELECTION--
> **The target label and input features are separated.**

In [6]:
X = df['clean_comment']
y = df['toxic']


### TF-IDF VECTORIZATION--
>**TF-IDF is used to convert text into numerical features.**

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=8000, ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(X)


### TRAIN-TEST SPLIT--
>**The dataset is split into training and testing sets.**

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)


### MODEL TRAINING--

> **Logistic Regression is used to train the classification model.**

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

###MODEL PREDICTION--
> **Predictions are made on the test dataset.**

In [11]:
y_pred = model.predict(X_test)


### MODEL EVALUATION---
> **The performance of the model is evaluated using standard metrics.**


In [12]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9295629014569952
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     28856
           1       0.59      0.86      0.70      3059

    accuracy                           0.93     31915
   macro avg       0.79      0.90      0.83     31915
weighted avg       0.95      0.93      0.94     31915



### CONFUSION MATRIX--
> **Confusion matrix is used to visualize prediction results.**

In [13]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


array([[27024,  1832],
       [  416,  2643]])

### TESTING WITH CUSTOM INPUT--
> **The model is tested using custom user input.**


In [14]:
def predict_comment(text):
    cleaned = clean_text(text)
    vector = vectorizer.transform([cleaned])
    return "Toxic" if model.predict(vector)[0] == 1 else "Non-Toxic"

predict_comment("You are stupid")


'Toxic'

### SAVE MODEL--

> **The trained model and vectorizer are saved for future use.**

In [16]:
import pickle

pickle.dump(model, open("/content/toxic_model.pkl", "wb"))
pickle.dump(vectorizer, open("/content/tfidf_vectorizer.pkl", "wb"))


In [17]:
!pip install streamlit pyngrok nltk scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.5 MB/s eta 0:00:00


In [18]:
%%writefile app.py
import streamlit as st
import pickle
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

model = pickle.load(open("/content/toxic_model.pkl", "rb"))
vectorizer = pickle.load(open("/content/tfidf_vectorizer.pkl", "rb"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return " ".join(text)

st.title("🛑 Toxic Comment Classification")

comment = st.text_area("Enter comment")

if st.button("Predict"):
    cleaned = clean_text(comment)
    vector = vectorizer.transform([cleaned])
    result = model.predict(vector)[0]

    if result == 1:
        st.error("🚨 Toxic Comment")
    else:
        st.success("✅ Non-Toxic Comment")


Writing app.py


In [21]:
from pyngrok import ngrok

ngrok.set_auth_token("30Mq4H1fnUQ9SmrHVr8osWNHVVz_5PH15PjPbEgFrxjbeQvS9")


In [22]:
!streamlit run app.py &>/content/logs.txt &


In [23]:
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://3722d4b7a50c.ngrok-free.app" -> "http://localhost:8501">